In [ ]:
%matplotlib inline

# Version 1 du modèle prédictif

On commence par importer les données dont on aura besoin

In [1]:
import pandas as pd

In [2]:
country = pd.read_csv('country.csv', index_col=0)
league = pd.read_csv('league.csv', index_col=0)
team = pd.read_csv('team.csv', index_col=0)
match = pd.read_csv('match.csv', index_col=0)
player = pd.read_csv('player.csv', index_col=0)
player_attributes = pd.read_csv('player_attributes.csv', index_col=0)

On retire les colonnes suivantes : B365H, B365D, B365A, BWH, BWD, BWA, IWH, IWD, IWA, LBH, LBD, LBA, PSH, PSD, PSA, WHH, WHD, WHA, SJH, SJD, SJA, VCH, VCD, VCA, GBH, GBD, GBA, BSH, BSD, BSA car je ne sais pas à quoi elles correspondent, il n'y a aucune indication

In [46]:
match.drop(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 
            'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'], 
           axis=1, inplace=True)

Dans cette version, on ne va pas s'interesser aux joeurs, on va rester au niveau de granularité supérieur, cad les équipes

In [51]:
match.drop(['home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 
            'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10',
            'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 
            'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 
            'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 
            'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 
            'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 
            'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 
            'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1',
            'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 
            'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
            'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 
            'away_player_9', 'away_player_10', 'away_player_11'],
          axis=1, inplace=True)

Enfin, on ne va pas non plus encore s'interesser aux corner, possession, tirs cadrés, etc. On les supprime aussi

In [62]:
match.drop(['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession'],
           axis=1, inplace=True)

In [63]:
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3


On va maintenant se limiter à la ligue 1 française dans un premier temps

In [64]:
match_ligue_1 = match[match.league_id == 4769]
num_teams = len(match_ligue_1.away_team_api_id.unique())
print(num_teams)

35


In [65]:
match_ligue_1.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
4768,4769,4769,4769,2008/2009,1,2008-08-09 00:00:00,483129,8583,9830,2,1
4769,4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,2,1
4770,4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,1,0
4771,4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,0,1
4772,4773,4769,4769,2008/2009,1,2008-08-10 00:00:00,483133,9748,9941,3,0


## Construction du modèle

Maintenant que les données sont propres et identifiées, on va construire le modèle pymc3

In [70]:
teams = match_ligue_1.home_team_api_id.unique()
teams = pd.DataFrame(teams, columns=['team'])
teams['i'] = teams.index

In [71]:
teams.head()

,team,i
0,8583,0
1,9827,1
2,9746,2
3,8682,3
4,9748,4


In [72]:
match_ligue_1 = pd.merge(match_ligue_1, teams, left_on='home_team_api_id', right_on='team', how='left')
match_ligue_1 = match_ligue_1.rename(columns = {'i': 'i_home'}).drop('team', 1)
match_ligue_1 = pd.merge(match_ligue_1, teams, left_on='away_team_api_id', right_on='team', how='left')
match_ligue_1 = match_ligue_1.rename(columns = {'i': 'i_away'}).drop('team', 1)

In [73]:
match_ligue_1.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,i_home,i_away
0,4769,4769,4769,2008/2009,1,2008-08-09 00:00:00,483129,8583,9830,2,1,0,18
1,4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,2,1,1,16
2,4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,1,0,2,13
3,4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,0,1,3,11
4,4773,4769,4769,2008/2009,1,2008-08-10 00:00:00,483133,9748,9941,3,0,4,15


In [74]:
observed_home_goals = match_ligue_1.home_team_goal.values
observed_away_goals = match_ligue_1.away_team_goal.values

home_team = match_ligue_1.i_home.values
away_team = match_ligue_1.i_away.values

On décrit notre modélisation

In [75]:
import pymc3 as pm

model = pm.Model()
with pm.Model() as model:
    # team-specific model parameters
    attaque = pm.HalfNormal("attaque", tau = 1, shape=num_teams)
    defense = pm.HalfNormal("defense", tau = 1, shape=num_teams)

    home_theta  = pm.math.exp(attaque[home_team] - defense[away_team])
    away_theta  = pm.math.exp(attaque[away_team] - defense[home_team])

    # likelihood of observed data
    home_points = pm.Poisson('home_points', mu=home_theta, observed=observed_home_goals)
    away_points = pm.Poisson('away_points', mu=away_theta, observed=observed_away_goals)

In [ ]:
with model:

    start = pm.find_MAP()
    step = pm.NUTS(state=start)
    trace = pm.sample(2000, step, init=start)

    pm.traceplot(trace)

         Current function value: 8515.602703
         Iterations: 85
         Function evaluations: 157
         Gradient evaluations: 146


 29%|██████████████████████▊                                                        | 579/2000 [17:13<09:51,  2.40it/s]

In [ ]:
pm.forestplot(trace, varnames=['atts'], ylabels=['France', 'Ireland', 'Scotland', 'Italy', 'England', 'Wales'], main="Team Offense")